# 그냥 스크래핑
10개 -> 3.2초  <br>
50개 -> 19.9초

In [65]:
import datetime
import requests
from bs4 import BeautifulSoup

class Solved():
    URL = 'https://solved.ac/search?query='
    selector = '#__next > div.contents > div:nth-child(3) > div:nth-child(2) > div > div.StickyTable__Wrapper-akg1ak-3.tcQcH.sticky-table > div > div:nth-child(2)'
    baek_URL = 'https://www.acmicpc.net/problem/'
    
    def __init__(self, prob_num):
        self.prob_num = str(prob_num)
    
    def get_tier(self):
        self.URL += self.prob_num
        self.baek_URL += self.prob_num
        res = requests.get(self.URL)
        soup = BeautifulSoup(res.text, 'html.parser')
        soup = soup.select_one(self.selector)
        soup_a = soup.find('a', {'href':self.baek_URL})
        tier_info = soup_a.img
        return tier_info

In [66]:
prob_nums = list(map(int, "1000 1008 1012 1021 1107 1149 1158 1182 1212 1248 1260 1309 1330 1373 1406 1463 1476 1620 1662 1676 1697 1699 1707 1712 1748 1759 1764 1874 1912 1918 1926 1927 1929 1932 1934 1935 1978 2004 2089 2091 2133 2146 2156 2167 2178 2193 2206 2217 2225 2231 2293 2294 2309 2468 2504 2529 2557 2577 2583 2588 2606 2609 2644 2667 2739 2742 2743 2745 2753 2869 2884 2941 3078 3085 4179 4949 5014 5427 6064 6588 6593 6603 7562 7569 7576 7785 8393 9012 9093 9095 9205 9375 9465 9498 9613 10026 10162 10171 10172 10430 10718 10773 10799 10807 10808 10809 10817 10818 10819 10820 10824 10828 10844 10845 10866 10869 10872 10951 10952 10971 10972 10973 10974 11005 11047 11052 11053 11054 11055 11057 11279 11576 11653 11655 11656 11722 11723 11724 11726 11727 11729 12865 13398 14002 14391 14500 14501 14888 14889 15649 15650 15651 15652 15654 15655 15656 15657 15658 15661 15663 15664 15665 15666 15988 15990 16194 16208 16956 17087 17103 17298 17299 17404 17413".split()))
print(prob_nums[:2])

[1000, 1008]


In [67]:
prob_tier_info_list = []
for prob_num in prob_nums[:10]:
    solved = Solved(prob_num)
    prob_tier_info_list.append(solved.get_tier().get('src'))
print(prob_tier_info_list)

['https://static.solved.ac/tier_small/1.svg', 'https://static.solved.ac/tier_small/2.svg', 'https://static.solved.ac/tier_small/9.svg', 'https://static.solved.ac/tier_small/7.svg', 'https://static.solved.ac/tier_small/11.svg', 'https://static.solved.ac/tier_small/10.svg', 'https://static.solved.ac/tier_small/6.svg', 'https://static.solved.ac/tier_small/9.svg', 'https://static.solved.ac/tier_small/2.svg', 'https://static.solved.ac/tier_small/13.svg']


# 멀티쓰레딩 적용
10개 -> 0.861초 <br>
50개 -> 2.4초

In [61]:
import requests
from bs4 import BeautifulSoup
import concurrent.futures

MAX_THREADS = 30

class Solved():
    URL = 'https://solved.ac/search?query='
    selector = '#__next > div.contents > div:nth-child(3) > div:nth-child(2) > div > div.StickyTable__Wrapper-akg1ak-3.tcQcH.sticky-table > div > div:nth-child(2)'
    baek_URL = 'https://www.acmicpc.net/problem/'
    
    def __init__(self, prob_nums):
        self.prob_nums = prob_nums
        self.tier_info = []
    
    def get_tier(self, prob_num):
        prob_URL = self.URL + prob_num
        baek_URL = self.baek_URL + prob_num
        res = requests.get(prob_URL)
        soup = BeautifulSoup(res.text, 'html.parser')
        soup = soup.select_one(self.selector)
        soup_a = soup.find('a', {'href':baek_URL})
        tier_info = soup_a.img
        self.tier_info.append(tier_info)
    
    def multi_threading(self):
        threads = min(MAX_THREADS, len(self.prob_nums))
        with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
            executor.map(self.get_tier, self.prob_nums[:50])
    
    def get_result(self):
        return self.tier_info

In [62]:
prob_nums = list("1000 1008 1012 1021 1107 1149 1158 1182 1212 1248 1260 1309 1330 1373 1406 1463 1476 1620 1662 1676 1697 1699 1707 1712 1748 1759 1764 1874 1912 1918 1926 1927 1929 1932 1934 1935 1978 2004 2089 2091 2133 2146 2156 2167 2178 2193 2206 2217 2225 2231 2293 2294 2309 2468 2504 2529 2557 2577 2583 2588 2606 2609 2644 2667 2739 2742 2743 2745 2753 2869 2884 2941 3078 3085 4179 4949 5014 5427 6064 6588 6593 6603 7562 7569 7576 7785 8393 9012 9093 9095 9205 9375 9465 9498 9613 10026 10162 10171 10172 10430 10718 10773 10799 10807 10808 10809 10817 10818 10819 10820 10824 10828 10844 10845 10866 10869 10872 10951 10952 10971 10972 10973 10974 11005 11047 11052 11053 11054 11055 11057 11279 11576 11653 11655 11656 11722 11723 11724 11726 11727 11729 12865 13398 14002 14391 14500 14501 14888 14889 15649 15650 15651 15652 15654 15655 15656 15657 15658 15661 15663 15664 15665 15666 15988 15990 16194 16208 16956 17087 17103 17298 17299 17404 17413".split())
print(prob_nums[:2])

['1000', '1008']


In [63]:
solved = Solved(prob_nums)
solved.multi_threading()
print(len(solved.get_result()))

50


# Baekjoon multithreading

In [ ]:
import datetime
import requests
import concurrent.futures
from bs4 import BeautifulSoup

class Baekjoon():
    member_URL = 'https://www.acmicpc.net/group/member/10060'
    member_selector = 'body > div.wrapper > div.container.content > div.row > div:nth-child(5)'
    solves_selector = '#status-table > tbody'
    
    def __init__(self):
        self.member_ids = []
        self.member_dict = {}
        self.MAX_THREADS = 30
        
    # crawl member ids
    def crawl_member_ids(self):
        res = requests.get(self.member_URL)
        soup = BeautifulSoup(res.text, 'html.parser')
        soup = soup.select_one(self.member_selector)
        # users info
        infos = soup.find_all('div', {'class': 'member'})
        for info in infos:
            self.member_ids.append(info.h4.a.text)    
    
    # crawl member solves for each id
    def crawl_member_solves(self, member_id):
        print(member_id)
        solves_URL = f'https://www.acmicpc.net/status?problem_id=&user_id={member_id}&language_id=-1&result_id=-1'
        res = requests.get(solves_URL)
        soup = BeautifulSoup(res.text, 'html.parser')
        soup = soup.select_one(self.solves_selector)
        solves_trs = soup.find_all('tr')
        
        # 맞은 문제는 적절한 데이터만 골라서 user_dict에 담기
        corrects = []
        for solves_tr in solves_trs:
            result_span = solves_tr.find('span', {'class': 'result-ac'})          
            if result_span:
                q = solves_tr.find('a', {'class': 'problem_title'})
                q_title = q.get('title')
                q_num = q.text
                solved_time = solves_tr.find('a', {'class': 'real-time-update'}).get('title')
                q_info_dict = {
                    'question_title': q_title,
                    'question_number': q_num,
                    'question_site': 'B',
                    'solved_time': solved_time,                
                }
                self.member_dict[member_id].append(q_info_dict)
    
    # assign threads to different ids
    def initiate_multithread_crawling(self, member_ids):
        # initiate member_dict
        for member_id in member_ids:
            self.member_dict[member_id] = []
        
        # initiate crawling with threads
        threads = min(self.MAX_THREADS, len(member_ids))
        with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
            executor.map(self.crawl_member_solves, member_ids)
    
    def get_member_dict(self):
        return self.member_dict        